In [1]:
import os
import jieba
import Spider

def highlight(item, query: str, side_len: int = 12) -> str:
    positions = list()
    query_words = list(jieba.cut(query))  # 把生成器强制转换为列表
    i = 0
    content_lower = item[2].lower()
    word_start_map = list()
    word_end_map = list()
    last_word_end = -1
    len_content_lower = len(content_lower)
    segments = list()
    for keyword in query_words:
        idx = content_lower.find(keyword.lower())
        positions.append(idx)
    for keyword in jieba.cut(content_lower):
        # 用于实现提取摘要时“整词切分”，避免出现截取摘要时首尾的词被截断
        current_word_start = last_word_end + 1
        current_word_end = current_word_start + len(keyword) - 1
        for _ in range(current_word_start, current_word_end+1):
            word_start_map.append(current_word_start)
            word_end_map.append(current_word_end)
        last_word_end = current_word_end
    positions.sort()
    while i < len(positions):
        start_pos = max(positions[i] - side_len, 0)
        end_pos = min(positions[i] + side_len, len_content_lower-1)
        # 用于实现合并相邻且有部分重合的摘要
        while (i < len(positions) - 1) and (positions[i+1] - positions[i] < side_len*2):
            end_pos = min(positions[i+1] + side_len, len_content_lower-1)
            i += 1
        start_ellipsis = '...' if start_pos > 0 else ''
        end_ellipsis = '...' if end_pos < len_content_lower else ''
        segments.append(start_ellipsis + item[2][word_start_map[start_pos]: word_end_map[end_pos]] + end_ellipsis)
        i += 1
    result = text = item[1] + '<br/>' + ''.join(segments)
    text_lower = text.lower()
    for keyword in query_words:
        # 高亮部分
        idx = text_lower.find(keyword.lower())
        if idx >= 0:
            ori_word = text[idx:idx+(len(keyword))]
            result = result.replace(ori_word, f'<span style="color:red";>{ori_word}</span>')
    return result


class MySearcherC10V0:
    """
    第九次课升级的搜索类版本：
    1、优化对多关键词查询的摘要获取
    2、优化对多关键词查询的高亮
    """
    def __init__(self, scale: int=1):
        self.docs = list()
        self.load_data()
        if scale > 1:
            self.docs *= scale  # 文档规模倍增，用于测试搜索速度
        self.cache = dict()
        self.vocab = set()
        self.lower_preprocess()
        jieba.load_userdict('./dict.txt')
        self.build_cache()

    def load_data(self, data_file_name='./news_list.pkl'):
        if os.path.exists(data_file_name):
            self.docs = Spider.pickle_load(data_file_name)
        else:
            Spider.pickle_save(data_file_name)
            self.docs = Spider.pickle_load(data_file_name)

    def search(self, query):
        result = None
        for keyword in jieba.cut(query.lower()):
            if keyword in self.cache:
                if result is None:
                    result = self.cache[keyword]
                else:
                    result = result & self.cache[keyword]
            else:
                result = set()
                break
        if result is None:
            result = set()
        sorted_result = self.rank(query, result)
        return sorted_result

    def rank(self, query, result_set):
        result = list()
        for doc_id in result_set:
            result.append([doc_id, self.score(self.docs[doc_id], query)])
        result.sort(key=lambda x: x[1], reverse=True)
        return result

    def render_search_result(self, query):
        count = 0
        result = ''
        for item in self.search(query):
            count += 1
            result += f'{count}[{item[1]}] {highlight(self.docs[item[0]], query)}\n'
        return result

    def score(self, item, query):
        score = 0
        # TODO 对query查询的分词避免重复
        for keyword in jieba.cut(query.lower()):
            title_score = item[1].lower().count(keyword.lower())
            content_score = item[2].lower().count(keyword.lower())
            score += title_score * 5 + content_score * 3
        return score

    def build_cache(self):
        """用分词（用文档过滤词库）的方式初始化缓存（构建索引）"""
        doc_id = 0
        for doc in self.docs:
            doc_word_set = set()
            for word in jieba.cut_for_search(doc[3]):
                if word not in doc_word_set:
                    result_item = doc_id
                    if word not in self.cache:
                        self.cache[word] = {result_item}
                    else:
                        self.cache[word].add(result_item)
                    self.vocab.add(word)
                    doc_word_set.add(word)
            doc_id += 1

    def lower_preprocess(self):
        for doc_id in range(len(self.docs)):
            self.docs[doc_id].append(
                (self.docs[doc_id][1] + ' ' + self.docs[doc_id][2]).lower())

    def simple_test(self):
        assert(len(self.search('tiktok')) > 1)

In [2]:
class MySearcherC10V1(MySearcherC10V0):
    """
    在score函数里添加停用词
    """
    def __init__(self, scale: int=1, stopwords_file:str = './stopwords.txt'):
        super().__init__(scale)
        self.stopwords = set()
        self.init_stopwords(stopwords_file)

    def init_stopwords(self, stopwords_file):
        with open(stopwords_file, 'r',encoding='UTF8') as f:
            self.stopwords = set(f.read().split())

    def score(self, item, query):
        score = 0
        for keyword in jieba.cut(query.lower()):
            weight = 1
            if keyword in self.stopwords:
                weight = 0.2
            title_score = item[1].lower().count(keyword.lower())
            content_score = item[2].lower().count(keyword.lower())
            score += (title_score * 5 + content_score * 3) * weight
        return score

In [7]:
%%time
searcher_v1 = MySearcherC10V1()

Wall time: 2.45 s


In [9]:
class MySearcherC10V2(MySearcherC10V0):
    """
    用文档频率对词进行加权
    """
    def __init__(self, scale: int=1):
        self.df = dict()
        super().__init__(scale)

    def build_cache(self):
        """
        用分词（用文档过滤词库）的方式构建索引
        """
        doc_id = 0
        for doc in self.docs:
            doc_word_set = set()
            for word in jieba.cut_for_search(doc[3]):
                if word not in doc_word_set:
                    result_item = doc_id
                    if word not in self.cache:
                        self.cache[word] = {result_item}
                    else:
                        self.cache[word].add(result_item)
                    self.vocab.add(word)
                    doc_word_set.add(word)
                    if word in self.df:
                        self.df[word] += 1
                    else:
                        self.df[word] = 1
            doc_id += 1

    def score(self, item, query):
        score = 0
        for keyword in jieba.cut(query.lower()):
            title_score = item[1].lower().count(keyword.lower())
            content_score = item[2].lower().count(keyword.lower())
            score += (title_score * 5 + content_score * 3) / len(item[2]) / self.df[keyword]
        return score

In [10]:
%%time
searcher_v2 = MySearcherC10V2()

Wall time: 1.63 s


In [11]:
print(searcher_v2.render_search_result('中国华为手机'))

1[0.00514408833827094] <span style="color:red";>华为</span>供应链公司：已向<span style="color:red";>华为</span>P50<span style="color:red";>手机</span>供货，供货时间有延后<br/>据<span style="color:red";>华为</span><span style="color:red";>手机</span>供应链公司，该公司已逐......品牌荣耀出售给了由30多家<span style="color:red";>中国</span>公司组成的财团，以帮...
2[0.0027402693446736907] 传<span style="color:red";>华为</span>预计今年智能<span style="color:red";>手机</span>出货量同比减少60%，降至7000万部<br/>...，据供应链消息人士透露，<span style="color:red";>华为</span>已通知其供应商，预计......7000万至8000万部智能<span style="color:red";>手机</span>的零部件。而且<span style="color:red";>华为</span>的零部......品牌荣耀出售给了由30多家<span style="color:red";>中国</span>公司组成的财团，以帮...
3[0.0023471858035917902] <span style="color:red";>华为</span>将在英国起诉汇丰，要拿到孟晚舟案关键文件<br/>...》消息，当地时间12日，<span style="color:red";>中国</span><span style="color:red";>华为</span>公司首席财务官孟晚舟......发生了很多事，有关于<span style="color:red";>华为</span><span style="color:red";>手机</span>出货量的报道，也有关...
4[0.0020744774812359295] 外媒：夺回<span style="color:red";>华为</span>失去的市场，荣耀仍能重现辉煌<